# Ex14


Um fundo de investimentos tem até R$ 300.000,00 para aplicar em duas ações. A empresa D é diversificada (tem 40% do seu capital aplicado em cerveja e o restante aplicado em refrigerantes) e espera-se que forneça bonificações de 12%. A empresa N não é diversificada (produz apenas cerveja) e espera-se que distribua bonificações de 20%. Para este investimento, considerando a legislação governamental aplicável, o fundo está sujeito às seguintes restrições:

- O investimento na empresa diversificada pode atingir até R$ 270.000,00.
- O investimento na empresa não-diversificada pode atingir até R$ 150.000,00.
- Em cada produto (cerveja ou refrigerante) pode-se investir até R$ 180.000,00.

Qual o esquema de investimento que maximiza o lucro?





# **Modelo Matemático do Problema de Investimento**

## **Variáveis de decisão**
Definição das variáveis:
- $x_D$ : investimento na empresa diversificada (D)
- $x_N$ : investimento na empresa não-diversificada (N)

## **Função objetivo**
Maximizar a bonificação total do investimento:

$$
\text{Maximize } Z = 0.12x_D + 0.20x_N
$$

## **Restrições**
1. **Orçamento total disponível**:
$$
x_D + x_N \leq 300000
$$

2. **Limite de investimento por empresa**:
$$
x_D \leq 270000
$$
$$
x_N \leq 150000
$$

3. **Limite de investimento por produto**:
   - **Investimento em Cerveja**:
$$
0.40x_D + x_N \leq 180000
$$

   - **Investimento em Refrigerante**:
$$
0.60x_D \leq 180000
$$

4. **Restrições de não negatividade**:
$$
x_D, x_N \geq 0
$$


In [11]:
import pyomo.environ as pyo

In [12]:
# Criação do modelo
model = pyo.ConcreteModel()

In [13]:

# Conjuntos

model.Empresas = pyo.Set(initialize=["D", "N"])

model.Produtos = pyo.Set(initialize=["Cerveja", "Refrigerante"])

In [14]:
# Bonificação esperada por empresa
bonificacao = {"D": 0.12, "N": 0.20}
model.bonificacao = pyo.Param(model.Empresas, initialize=bonificacao)

# Porcentagem de cada produto na empresa D
porcentagem_produto = {
    ("D", "Cerveja"): 0.40,
    ("D", "Refrigerante"): 0.60,
    ("N", "Cerveja"): 1.00
}
model.p_produto = pyo.Param(model.Empresas, model.Produtos, initialize=porcentagem_produto, default=0)

# Limite de investimento por empresa
limite_empresa = {"D": 270000, "N": 150000}
model.l_empresa = pyo.Param(model.Empresas, initialize=limite_empresa)

# Limite de investimento por produto
limite_produto = {"Cerveja": 180000, "Refrigerante": 180000}
model.l_produto = pyo.Param(model.Produtos, initialize=limite_produto)

# Orçamento total disponível
model.orcamento = pyo.Param(initialize=300000)


In [15]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Empresas, domain=pyo.NonNegativeReals)

In [16]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[e] * model.bonificacao[e] for e in model.Empresas)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [17]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Restrição de orçamento total
def orcamento_rule(model):
    return sum(model.x[e] for e in model.Empresas) <= model.orcamento
model.orcamento_total = pyo.Constraint(rule=orcamento_rule)

# 2. Restrição de limite de investimento por empresa
def limite_empresa_rule(model, e):
    return model.x[e] <= model.l_empresa[e]


model.limite_empresa = pyo.Constraint(model.Empresas, rule=limite_empresa_rule)

# 3. Restrição de limite de investimento por produto
def limite_produto_rule(model, p):
    return sum(model.x[e] * model.p_produto[e, p] for e in model.Empresas) <= model.l_produto[p]

model.limite_produto = pyo.Constraint(model.Produtos, rule=limite_produto_rule)




In [18]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex14.lp", io_options={"symbolic_solver_labels": True})

('ex14.lp', 2645062103152)

In [19]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 5 rows; 2 cols; 7 nonzeros
Coefficient ranges:
  Matrix [4e-01, 1e+00]
  Cost   [1e-01, 2e-01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+05, 3e+05]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-3); columns 2(-0); elements 4(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          2    -4.4000000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  4.4000000000e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 44.000,00


In [20]:
# Exibindo os resultados do modelo
print("Resultados do Modelo:")
for e in model.Empresas:
    valor = pyo.value(model.x[e])
    print(f"Investimento na empresa {e}: R$ {valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))



Resultados do Modelo:
Investimento na empresa D: R$ 200.000,00
Investimento na empresa N: R$ 100.000,00
